https://adventofcode.com/2022/day/19

In [30]:
import re
import operator
from functools import reduce

In [2]:
with open("data/19.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian
"""

In [4]:
def load_data(data):
    ORE, CLY, OBS, GEO = [0, 1, 2, 3]
    L = []
    for line in data.strip().splitlines():
        bp = [[0, 0, 0, 0] for _ in range(4)]
        _, bp[ORE][ORE], bp[CLY][ORE], bp[OBS][ORE], bp[OBS][CLY], bp[GEO][ORE], bp[GEO][OBS] = (int(x) for x in re.findall(r"\d+", line))
        L.append(bp)

    return L

In [34]:
bps = load_data(testdata)
bps[0]

[[4, 0, 0, 0], [2, 0, 0, 0], [3, 14, 0, 0], [2, 0, 7, 0]]

In [36]:
def dfs(bp, max_time=24):
    inds = ORE, CLY, OBS, GEO = [0, 1, 2, 3]
    max_minerals = [0, 0, 0, 0]
    for bot in inds:
        max_minerals[bot] = max(req[bot] for req in bp)

    visited = {}
    stack = [((1,), max_time, [1, 0, 0, 0], [0, 0, 0, 0])]
    max_geo = 0
    while stack:
        pth, tr, bs, ms = stack.pop()
        if tr <= 0:
            continue
        max_geo = max(max_geo, ms[GEO] + bs[GEO] * tr)
        if pth in visited:
            continue
        visited[pth] = (tr, bs, ms)
        for bi in inds:
            # do we need this bot?
            if bi != GEO and max_minerals[bi] * tr <= ms[bi] + bs[bi] * tr:
                visited[pth + (bi,)] = -1
                continue

            # Do precursor bots exist yet?
            reqs = bp[bi]
            if not all(x for x,y in zip(bs, reqs) if y):
                visited[pth + (bi,)] = -2
                continue

            tr1, bs1, ms1 = tr, bs.copy(), ms.copy()
            # If not all required minerals are present, wait for them to be produced
            while any(a-b < 0 for a,b in zip(ms1, reqs)):
                tr1 -= 1
                ms1 = [a+b for a,b in zip(bs1, ms1)]
            ms1 = [a-b for a,b in zip(ms1, reqs)]
            tr1 -= 1
            ms1 = [a+b for a,b in zip(bs1, ms1)]
            bs1[bi] += 1
            stack.append((pth+(bi,), tr1, bs1, ms1))

    return max_geo


In [37]:
%%time
dfs(bps[0])

CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.34 s


9

In [38]:
%%time
dfs(bps[1])

CPU times: user 763 ms, sys: 24 µs, total: 763 ms
Wall time: 761 ms


12

In [39]:
%%time
bps = load_data(data)
ttl = 0
for bpid, bp in enumerate(bps, 1):
    geodes = dfs(bp)
    ttl += bpid * geodes
print(ttl)

1389
CPU times: user 6.24 s, sys: 3.89 ms, total: 6.24 s
Wall time: 6.24 s


In [40]:
%%time
L = []
for bp in bps[:3]:
    geodes = dfs(bp, max_time=32)
    print(geodes)
    L.append(geodes)

11
13
21
CPU times: user 2min 14s, sys: 5.01 s, total: 2min 19s
Wall time: 2min 19s


In [41]:
reduce(operator.mul, L)

3003